In [2]:
import os
import math
import numpy as np
import ffmpeg
import cv2
import imutils
from imutils.video import count_frames
import matplotlib.pyplot as plt
from IPython.display import Audio

import stitching

from stitching.image_handler import ImageHandler
from stitching.feature_detector import FeatureDetector
from stitching.feature_matcher import FeatureMatcher
from stitching.subsetter import Subsetter
from stitching.camera_estimator import CameraEstimator
from stitching.camera_adjuster import CameraAdjuster
from stitching.camera_wave_corrector import WaveCorrector
from stitching.warper import Warper
from stitching.cropper import Cropper
from stitching.seam_finder import SeamFinder
from stitching.exposure_error_compensator import ExposureErrorCompensator
from stitching.blender import Blender
from stitching.timelapser import Timelapser

In [3]:
ROOT_DIR = os.path.dirname(os.getcwd())
DATA_FOLDER = os.path.join(ROOT_DIR, "data")

In [4]:
video_keparoicam_L = os.path.join(DATA_FOLDER, "keparoicam_clipL_synchronized.mp4")
video_keparoicam_R = os.path.join(DATA_FOLDER, "keparoicam_clipR_synchronized.mp4")

In [6]:
video_left_capture = cv2.VideoCapture(video_keparoicam_L)
video_right_capture = cv2.VideoCapture(video_keparoicam_R)

## Calculate intermediate objects for panorama stitching

In [13]:
ret_left, left_frame = video_left_capture.read()
ret_right, right_frame = video_right_capture.read()

In [17]:
images = [left_frame, right_frame]

left_path = os.path.join("/tmp" , 'stitcher_left_image.jpg')
right_path = os.path.join("/tmp" , 'stitcher_right_image.jpg')

cv2.imwrite(left_path, left_frame)
cv2.imwrite(right_path, right_frame)
image_names = [left_path, right_path]

In [18]:
img_handler = ImageHandler()
finder = FeatureDetector(detector="sift")

In [19]:
img_handler.set_img_names(image_names)

medium_imgs = list(img_handler.resize_to_medium_resolution())
low_imgs = list(img_handler.resize_to_low_resolution(medium_imgs))
final_imgs = list(img_handler.resize_to_final_resolution())

In [20]:
features = [finder.detect_features(img) for img in medium_imgs]
keypoints_center_img = finder.draw_keypoints(medium_imgs[1], features[1])